In [1]:
import os, sys
sys.path.insert(1, '../')
import concentration as conc

import numpy as np
import scipy as sc
import pandas as pd

from importlib import reload
import utils

import matplotlib
import matplotlib.patheffects as pe
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
d = np.load('results-finite-population-111022.npz')
mu1maici_pxnxtx2 = d['mu1maici_pxnxtx2']
mu1claci_pxnxtx2 = d['mu1claci_pxnxtx2']
mu0maici_pxnxtx2 = d['mu0maici_pxnxtx2']
mu0claci_pxnxtx2 = d['mu0claci_pxnxtx2']
omaici_pxnxtx2 = d['omaici_pxnxtx2']
oclaci_pxnxtx2 = d['oclaci_pxnxtx2']

In [ ]:
# compute true odds ratio

In [ ]:
# construct naive confidence interval

In [ ]:
idx = 106
theta_star = np.quantile(native_df.measured, 0.5)
print(theta_star)
ci_pp = [pp_df.iloc[idx].lower, pp_df.iloc[idx].upper]
ci_cl = [cl_df.iloc[idx].lower, cl_df.iloc[idx].upper]

plt.figure(figsize=(6.4,4.8))
plt.plot(
    ci_pp,[0.4,0.4],
    linewidth=20, color="#DAF3DA",
    path_effects=[pe.Stroke(linewidth=22, foreground="#71D26F"), pe.Normal()],
    label=' prediction-powered'
)
plt.plot(
    ci_cl, [0.2, 0.2],
    linewidth=20, color="#EEEDED",
    path_effects=[pe.Stroke(linewidth=22, foreground="#BFB9B9"), pe.Normal()],
    label=' classical'
)
plt.plot(
    ci_naive, [0.2, 0.2],
    linewidth=20, color="#fbdbc4",
    path_effects=[pe.Stroke(linewidth=22, foreground="#F7AE7C"), pe.Normal()],
    label=' naive'
)
plt.plot(
    [theta_star, theta_star], [0, 0.5], linestyle='--', dashes=(2, 1), linewidth=3,
    label="true median", color="#bbbdf5"
)
plt.xlabel("expression level", fontsize=20)
plt.yticks([], labels=[])
plt.xticks([5, 6, 7, 8, 9], fontsize=20)
plt.ylim([0,1])
leg = plt.legend(fontsize=20, borderpad=1, labelspacing = 0.75, bbox_to_anchor=[1.1,0.5])
sns.despine(ax=plt.gca(),top=True,right=True,left=True)
# plt.gcf().subplots_adjust(right=0.85)

plt.savefig('./figures/intervals.pdf', ppi=300, bbox_inches='tight')
plt.show()